In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms



city_train = datasets.Cityscapes('./data/', split='train', mode='fine', transform=transforms.ToTensor(), target_type='semantic')
city_test = datasets.Cityscapes('./data/', split='test', mode='fine', transform=transforms.ToTensor(), target_type='semantic')
city_val = datasets.Cityscapes('./data/', split='val', mode='fine', transform=transforms.ToTensor(), target_type='semantic')

img, smnt = city_train[0]

RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory